<!--NAVIGATION-->
< [Working with Time Series](03.11-Working-with-Time-Series.ipynb) | [Contents](Index.ipynb) | [Further Resources](03.13-Further-Resources.ipynb) >

# 3.13 高性能Pandas: eval()与query()

前面的章节已经介绍过，Python数据科学生态环境的强大力量建立在NumPy与Pandas的基础之上，并通过直观的语法将基本操作转换成C语言：在NumPy里是向量化/广播运算，在Pandas里是分组型的运算。

虽然这些抽象功能可以简洁高效地解决许多问题，但是它们经常需要创建临时中间对象，这样就会占用大量的计算时间与内存。

Pandas从0.13版开始（2014年1月）就引入了实验性工具，让用户可以直接运行C语言速度的操作，不需要十分费力地配置中间数组。它们就是``eval()``和``query()``函数，都依赖于[Numexpr](https://github.com/pydata/numexpr)程序包。我们将在下面的Notebook中演示其用法，并介绍一些使用时的注意事项。

## 1. ``query()``与``eval()``的设计动机： 复合代数式

前面已经介绍过，NumPy与Pandas都支持快速的向量化运算。例如，你可以对下面两个数组进行求和：

In [1]:
import numpy as np
rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)
%timeit x + y

6.05 ms ± 77.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


这样做比普通的Python循环或列表综合要快很多：

In [2]:
%timeit np.fromiter((xi + yi for xi, yi in zip(x, y)), dtype=x.dtype, count=len(x))

431 ms ± 9.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


但是这种运算在处理复合代数式（compound expression）问题时的效率比较低，例如下面的表达式：

In [3]:
mask = (x > 0.5) & (y < 0.5)

由于NumPy会计算每一个代数子式，因此这个计算过程等价于：

In [4]:
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

也就是说，每段中间过程都需要显式地分配内存。如果``x``数组和``y`` 数组非常大，这么运算就会占用大量的时间和内存消耗。 

Numexpr程序库可以让你在不为中间过程分配全部内存的前提下，完成元素到元素的复合代数式运算。虽然[Numexpr documentation](https://github.com/pydata/numexpr)程序库里提供了更详细的内容，但是简单点儿说，这个程序库其实就是用一个NumPy风格的字符串代数式进行运算：

In [5]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr) #np.allclose比较两个array是不是每一元素都相等,默认在1e-05的误差范围内

ModuleNotFoundError: No module named 'numexpr'

这么做的好处是，由于Numexpr在计算代数式时不需要为临时数组分配全部内存，因此计算比NumPy更高效，尤其适合处理大型数组。马上要介绍的Pandas的``eval()``和``query()``工具其实也是基于Numexpr实现的。

## 2. 用``pandas.eval()``实现高性能运算

Pandas的``eval()``函数用字符串代数式实现了``DataFrame``的高性能运算，例如下面的``DataFrame``：

In [6]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                      for i in range(4))

如果要用普通的Pandas方法计算四个``DataFrame``的和，可以这么写：

In [7]:
%timeit df1 + df2 + df3 + df4

255 ms ± 52.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


也可以通过``pd.eval``和字符串代数式计算并得出相同的结果：

In [9]:
%timeit pd.eval('df1 + df2 + df3 + df4')

209 ms ± 4.73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


``pd.eval``版本的代数式比普通方法快一倍（而且内存消耗更少），结果也是一样的：

In [10]:
np.allclose(df1 + df2 + df3 + df4,
            pd.eval('df1 + df2 + df3 + df4'))

True

### ``pd.eval()``支持的运算

从Pandas v0.16版开始，``pd.eval()``就支持许多运算了。为了演示这些运算，创建一个整数类型的``DataFrame``：

In [11]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                           for i in range(5))

#### (1) 算术运算符

``pd.eval()``支持所有的算术运算符，例如：

In [12]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

#### (2) 比较运算符

``pd.eval()``支持所有的比较运算符，包括链式代数式（chained expression）

In [13]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

#### (3) 位运算符

``pd.eval()``支持``&``（与）和``|``（或）等位运算符：

In [14]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

另外，你还可以在布尔类型的代数式中使用``and``和``or``等字面值：

In [15]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

#### (4) 对象属性与索引

``pd.eval()``可以通过``obj.attr``语法获取对象属性，通过``obj[index]``语法获取对象索引：

In [16]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

#### (5) 其他运算

目前``pd.eval()``还**不支持**函数调用、条件语句、循环以及更复杂的运算。如果你想要进行这些运算，可以借助Numexpr来实现。

## 3. 用``DataFrame.eval()``实现列间运算

由于``pd.eval()``是Pandas的顶层函数，因此``DataFrame``有一个``eval()``方法可以做类似的运算。使用``eval()``方法的好处是可以借助列名称进行运算，示例如下：

In [17]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


如果用前面介绍的``pd.eval()``，就可以通过下面的代数式计算这三列：

In [18]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

而``DataFrame.eval()``方法可以通过列名称实现简洁的代数式：

In [19]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

请注意，这里用**列名称作为变量**来计算代数式，结果同样是正确的。

### 1. 用``DataFrame.eval()``新增列

除了前面介绍的运算功能，``DataFrame.eval()``还可以创建新的列。还用前面的``DataFrame``来演示，列名是``'A'``, ``'B'``和``'C'``:

In [20]:
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


可以用``df.eval()``创建一个新的列``'D'``，然后赋给它其他列计算的值：

In [21]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


还可以修改已有的列：

In [22]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


## 4. ``DataFrame.query()``方法

``DataFrame``基于字符串代数式的运算实现了另一个方法，被称为``query()``，例如：

In [24]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

和前面介绍过的``DataFrame.eval()``一样，这是一个用``DataFrame``列创建的代数式，但是不
能用``DataFrame.eval()``语法(因为你要的结果是包含DataFrame 的全部列)。不过，对于这种过滤运算，你可以用``query()``方法：

In [25]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

## 5. 性能决定使用时机

在考虑要不要用这两个函数时，需要思考两个方面：**计算时间**和**内存消耗**，而内存消耗是
更重要的影响因素。就像前面介绍的那样，每个涉及NumPy数组或Pandas的``DataFrame``
的复合代数式都会产生临时数组，例如：

In [26]:
x = df[(df.A < 0.5) & (df.B < 0.5)]

它基本等价于：

In [27]:
tmp1 = df.A < 0.5
tmp2 = df.B < 0.5
tmp3 = tmp1 & tmp2
x = df[tmp3]

如果临时``DataFrame``的内存需求比你的系统内存还大（通常是几吉字节），那么最好还是使用``eval()``和``query()``代数式。你可以通过下面的方法大概估算一下变量的内存消耗：

In [28]:
df.values.nbytes

32000

在性能方面，即使你没有使用最大的系统内存，``eval()``的计算速度也比普通方法快。现在的性能瓶颈变成了临时``DataFrame``与系统CPU的L1和L2缓存（在2016年依然是几兆字节）之间的对比了——如果系统缓存足够大，那么``eval()``就可以避免在不同缓存间缓慢地移动临时文件。在实际工作中，我发现普通的计算方法与``eval/query``计算方法在计算时间上的差异并非总是那么明显，普通方法在处理较小的数组时反而速度更快！``eval/query``方法的优点主要是节省内存，有时语法也更加简洁。

我们已经介绍了``eval()``与``query()``的绝大多数细节，若想了解更多的信息，请参考Pandas文档。尤其需要注意的是，可以通过设置不同的解析器和引擎来执行这些查询，相关细节请参考 Pandas文档中["Enhancing Performance" section](http://pandas.pydata.org/pandas-docs/dev/enhancingperf.html)节。

<!--NAVIGATION-->
< [Working with Time Series](03.11-Working-with-Time-Series.ipynb) | [Contents](Index.ipynb) | [Further Resources](03.13-Further-Resources.ipynb) >